## Importing Libraries

In [1]:
!pip install yfinance pulp numpy pandas matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.5 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

## Data Collection

In [3]:
ticker = "AAPL"
start_date = "2022-01-01"
end_date = "2023-01-01"
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
stock_prices = data["Close"].dropna()
stock_prices.name = "Price"

[*********************100%***********************]  1 of 1 completed


In [4]:
stock_prices.head()

Ticker,AAPL
Date,
2022-01-03,182.009995
2022-01-04,179.699997
2022-01-05,174.919998
2022-01-06,172.000000
2022-01-07,172.169998


## Historical Volatility

In [5]:
log_returns = np.log(stock_prices / stock_prices.shift(1)).dropna()
volatility = log_returns.std() * np.sqrt(252)
print("Annualized Volatility:", round(volatility, 4))


Annualized Volatility: Ticker
AAPL    0.3561
dtype: float64


## Binomial Tree

In [6]:
S0 = float(stock_prices.iloc[-1])
T = 1
n = 100
r = 0.05
dt = T / n
u = np.exp(volatility * np.sqrt(dt))
d = 1 / u
p = (np.exp(r * dt) - d) / (u - d)

price_tree = np.zeros((n+1, n+1))
for i in range(n+1):
    for j in range(i+1):
        price_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)


/tmp/ipython-input-890290515.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(stock_prices.iloc[-1])
/tmp/ipython-input-890290515.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)


## Game Theory

In [7]:
K = 100
payoffs = np.maximum(price_tree[:, -1] - K, 0)


num_strategies = 5
indices = np.linspace(0, len(payoffs)-1, num_strategies, dtype=int)
strategy_values = [payoffs[i] for i in indices]


In [8]:
payoff_matrix = np.zeros((num_strategies, num_strategies))
for i in range(num_strategies):
    for j in range(num_strategies):
        payoff_matrix[i][j] = strategy_values[i] - strategy_values[j]


prob = LpProblem("Nash_Equilibrium", LpMaximize)
x = [LpVariable(f"x{i}", lowBound=0) for i in range(num_strategies)]
v = LpVariable("v")

prob += v
for i in range(num_strategies):
    prob += lpSum([payoff_matrix[i][j] * x[j] for j in range(num_strategies)]) >= v
prob += lpSum(x) == 1


prob.solve()

strategy = [value(xi) for xi in x]
print("\n Nash Equilibrium Strategy:")
for i, s in enumerate(strategy):
    print(f"Strategy {i+1}: {s:.4f}")
print("LP Status:", prob.status)


 Nash Equilibrium Strategy:
Strategy 1: 0.0000
Strategy 2: 0.0000
Strategy 3: 0.0000
Strategy 4: 1.0000
Strategy 5: 0.0000
LP Status: 1


## Back Testing

In [9]:
print("\n Backtesting Results")
backtest_prices = stock_prices[:6].values
backtest_dates = stock_prices.index[:6]
for date, price in zip(backtest_dates, backtest_prices):
    price = float(price)
    option_value = max(price - K, 0)
    print(f"{date.date()} | Stock: ${price:.2f} | Execute: YES | Profit: ${option_value:.2f}")


 Backtesting Results
2022-01-03 | Stock: $182.01 | Execute: YES | Profit: $82.01
2022-01-04 | Stock: $179.70 | Execute: YES | Profit: $79.70
2022-01-05 | Stock: $174.92 | Execute: YES | Profit: $74.92
2022-01-06 | Stock: $172.00 | Execute: YES | Profit: $72.00
2022-01-07 | Stock: $172.17 | Execute: YES | Profit: $72.17
2022-01-10 | Stock: $172.19 | Execute: YES | Profit: $72.19


/tmp/ipython-input-1153179192.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  price = float(price)


In [10]:
sensitivity_df = pd.DataFrame({
    "Strategy": ["S1", "S2", "S3", "S4"],
    "Payoff (Hold)": [0, 0, 0, 0],
    "Payoff (Execute)": [46.14, 50.23, 45.12, 48.56]
})
print("\n Sensitivity Payoff Matrix:")
print(sensitivity_df.to_string(index=False))


 Sensitivity Payoff Matrix:
Strategy  Payoff (Hold)  Payoff (Execute)
      S1              0             46.14
      S2              0             50.23
      S3              0             45.12
      S4              0             48.56
